In [2]:
import numpy as np
import pandas as pd

import yfinance as yf
from datetime import datetime, date

import numpy as np
from numpy.ma.core import ceil
from scipy.spatial import distance #distance calculation
from sklearn.preprocessing import MinMaxScaler #normalisation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score #scoring
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib import animation, colors

from minisom import MiniSom

# Função para classificar os dados 
A diferença

In [3]:
def f(x):
  if x < -4.5/100:
    return 0
  elif -4.5/100 < x and x < -2.5/100:
    return 1
  elif -2.5/100 < x and x < -0.5/100:
    return 2
  elif -0.5/100 < x and x < 0.5/100:
    return 3
  elif 0.5/100 < x and x < 2.5/100:
    return 4
  elif 2.5/100 < x and x < 4.5/100:
    return 5
  else:
    return 6


# Obtendo Dados

In [ ]:
# Download dos dados da B3
df = yf.Ticker('PETR4.SA').history(period='1d', start=datetime(2022,1,1), end=datetime(2022,12,28))[['Close', 'Open', 'High', 'Volume']]
df.reset_index(inplace=True) # resetar indeces
df['perc'] = df['Close'].pct_change(periods=2) # calculo de porcentagem da diferenca entre o o dia corrente e o proximo
df['interv'] = df['perc'].apply(f) # indice do intervalor dado
df = df.dropna() # retirar dados NaN
df

In [5]:
data_x = df[['Close', 'Open', 'Open', 'High', 'Volume']].to_numpy()
data_y = df['interv']

In [6]:
# train and test split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape) # check the shapes

(196, 5) (196,) (50, 5) (50,)


## Hiperparâmtros

In [ ]:
x = len(X_train)
y = len(X_train[0])
sigma = 0.3
alpha = 0.25

print(x, y)

# Rede Self Organized Map

In [ ]:
def classification_report(som, data):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    winmap = som.labels_map(train_x, train_y)
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in data:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

som = MiniSom(x,y, 5, sigma=sigma, learning_rate=alpha)
som.pca_weights_init(train_x)
som.train_random(train_x, 1000, verbose=False)

print(classification_report(test_y, classify(som, test_x)))


# Koniokortex Like Network

In [ ]:
incw = np.zeros((y,1))
alpha = 0.001 # fator de aprendizado
shift = 0.05*np.ones((y,1)) # deslocamento da sigmoide
v_des = 0.025 # velocidade do deslocamento
w = np.random.normal(size=(5, 1))
steps = 1000

# print(incw, alpha, shift, w)

In [ ]:
o = np.zeros((x,1))
o_antes = o.copy()

print(o, o_antes)

In [ ]:
# aprendizado do KLN
for j in range(x):
  w_old = w.copy()
  w = w + incw

  norm = np.linalg.norm(X_train[j,:])

  X_train_copy = X_train[j].copy()/norm

  w = w * o[j,:]


  # atualizacao do incremento
  incw = alpha * (o * o_antes - ((1+0.05)* np.ones((x,1))))# * o_antes) * w))
  shift = (v_des * o + shift) / (1 + v_des)# atualizacao do deslocamento

  o_antes = o.copy()

print("Rede KLN treinada")

In [ ]:
print(classification_report(test_y, classify(w, test_x)))